# Processor application

This notebook is the heart of the application which receives the content through a Kafka stream and returns the age rating. 

After receiving the data the model is used to predict the rating of that part of the text and return send a message with the rating so another Kafka consumer can use it.

In [33]:
# Add common folder to pythonpath
import sys
import numpy as np
sys.path
sys.path.append('/home/jovyan/work/common')

In [34]:
from sklearn.feature_extraction.text import HashingVectorizer

In [35]:
import os
import json
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [36]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from kafka import KafkaProducer

In [37]:
APP_NAME = "SubtitleRatingProcessor"

try:
    sc.stop()
    ssc.stop()
except:
    pass
    
sc = SparkContext(appName=APP_NAME)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, 5)
sc.addFile("/home/jovyan/work/common/preprocessor.py")

In [38]:
kafkaStream = KafkaUtils.createDirectStream(
    ssc,
    ['asd'],
    {"metadata.broker.list": 'kafka:9092'}
)

In [39]:
parsed = kafkaStream

In [40]:
import pickle

filename = 'model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

def message_to_vector(msg):
    text = msg[1]
    vectorizer = HashingVectorizer(n_features=2**4)
    X = vectorizer.fit_transform([text])
    return X

def vector2pred(msg):
    # return model.predict(msg)
    result = loaded_model.predict(msg.reshape(1, -1))
    print(result)
    return result
#     return np.average(msg.todense())
    
def sendPred(msg):
    producer = KafkaProducer(bootstrap_servers='kafka')
    producer.send("asd1", f"{msg}".encode('utf-8'))
    producer.flush()
    return msg

parsed.map(message_to_vector).map(vector2pred).map(sendPred).pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

In [22]:
ssc.stop()
sc.stop()